In [ ]:
# Jupyter notebook to import 4 files generated from Twitter Data Toolkit, extract nested data into new rows, and merge all data into one CSV/Excel file. Requires Python 3.x, Numpy and Pandas. Messy but it works, I'll create a function for this later on. 

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Import 4 files created by Twitter Data Toolkit
new_users = pd.read_json(r"users.json")
ext_tweets = pd.read_json(r"ext-tweets.json")
places = pd.read_json(r"places.json")
tweets = pd.read_json(r"tweets.json")

In [ ]:
# Users: Merge and extract nested data, drop irrelevant rows, rename new rows. 
for index, row in new_users.iterrows():
    network = row['public_metrics']
    
    # Extract followers.
    new_users.at[index,'followers'] = network['followers_count']

    # Extract friends.
    new_users.at[index,'following'] = network['following_count']
    
    # Extract total tweets.
    new_users.at[index,'total_tweets'] = network['tweet_count']
    
new_users = new_users.drop(['public_metrics','entities','pinned_tweet_id','profile_image_url','withheld'], axis=1)

new_users = new_users.rename(columns={"created_at": "profile_created", "description": "profile_desc", "location":"profile_loc", "id": "author_id"})

new_users.head()

In [ ]:
# Extended Tweets: Merge and extract nested data, drop irrelevant rows, rename new rows.
for index, row in ext_tweets.iterrows():
    network = row['public_metrics']
    
    # Extract RTs.
    ext_tweets.at[index,'retweets'] = network['retweet_count']
    
    # Extract replies.
    ext_tweets.at[index,'replies'] = network['reply_count']
    
    # Extract replies.
    ext_tweets.at[index,'likes'] = network['like_count']
    
    # Extract replies.
    ext_tweets.at[index,'quotes'] = network['quote_count']
    

ext_tweets = ext_tweets.drop(['attachments','conversation_id','entities','lang','public_metrics','referenced_tweets','withheld','created_at'], axis=1, errors='ignore')

ext_tweets = ext_tweets.rename(columns={"geo": "tweet_loc", "text": "full_text", "id":"tweet_id"})

In [ ]:
# Places: Merge and extract nested data, rename new rows.
places = places.drop(['country_code','geo'], axis=1, errors='ignore')
places = places.rename(columns={"full_name": "tweet_loc_long", "name": "tweet_loc_short", "id": "geo_id"})

In [ ]:
# Tweets: Merge and extract nested data, rename new rows.
for index, row in tweets.iterrows():
    network = row['public_metrics']
    
    # Extract RTs.
    tweets.at[index,'retweets'] = network['retweet_count']
    
    # Extract replies.
    tweets.at[index,'replies'] = network['reply_count']
    
    # Extract replies.
    tweets.at[index,'likes'] = network['like_count']
    
    # Extract replies.
    tweets.at[index,'quotes'] = network['quote_count']

tweets = tweets.drop(['lang','public_metrics','withheld'], axis=1, errors='ignore')
tweets = tweets.rename(columns={'geo': 'geo_id', 'id': 'tweet_id'})

# Remove irrelevant characters to match geo ID. 
tweets['geo_id'] = tweets['geo_id'].astype(str)
tweets['geo_id'] = tweets['geo_id'].str[14:-2]

In [ ]:
# Merge tweets with places.
tweets = pd.merge(tweets, places, how='left', on='geo_id')

# Merge tweets with extended tweets.
tweets = pd.merge(tweets, ext_tweets, how='left', on='tweet_id')

# Drop irrelevant columns before large merge with users.
tweets = tweets.rename(columns={'author_id_x': 'author_id'})
tweets = tweets.drop(['author_id_y','in_reply_to_user_id_y','possibly_sensitive_y','reply_settings_y','source_y','retweets_y','replies_y','likes_y','quotes_y', ], axis=1, errors='ignore')

# Drop duplicates to speed up tweets + users merge.
tweets = tweets.drop_duplicates(subset=['tweet_id'])
new_users = new_users.drop_duplicates(subset=['author_id'])

tweets = pd.merge(tweets, new_users, how='left', on='author_id')

In [ ]:
# Drop irrelevant columns that would have generated after merge with users.
tweets = tweets.drop(['author_id_y','in_reply_to_user_id_y','possibly_sensitive_y','reply_settings_y','source_y','retweets_y','replies_y','likes_y','quotes_y', ], axis=1, errors='ignore')

In [ ]:
# Drop in case any duplicates remain.
tweets = tweets.drop_duplicates(subset=['tweet_id'])

In [ ]:
# See total number of cleaned tweets.
len(tweets)

In [ ]:
# See a sample of tweets and rows to ensure it looks correct.
tweets.head()

In [ ]:
# Save as Excel file.
tweets.to_excel("combined-tweets.xlsx")

# Save as CSV file. Uncomment if this is the preferred option.
# tweets.to_csv("combined-tweets.csv")